# 填充和步幅

看一下是如何使用这两个东西的，首先看填充

例如，在所有侧边填充一个像素

In [4]:
import torch
from torch import nn

def comp_conv2d(conv2d,X):
    X=X.reshape((1,1)+X.shape)
    # 由于这里的X没有考虑通道数和批量大小数，所以这里在X.shape前面加了两个1
    # 表示通道数为1，批量大小数为1
    Y=conv2d(X)
    return Y.reshape(Y.shape[2:])
    # 把通道和批量数删除了，只要后面的形状

conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1)
X=torch.rand(size=(8,8))

comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

+ 注意，这里的padding是指一侧的padding，框架一般都是指定一侧的padding，而沐神课上讲的padding是两层一共的padding，矩阵输入的长/宽实际直接增大的个数
+ 所以整体来说，$k_h-2p_h=1$时，卷积层的输入和经过卷积的输出大小不变。没问题

虽然一般都会让填充的上下左右都一样，但是也可以不一样，例如

In [6]:
conv2d=nn.Conv2d(1,1,kernel_size=(5,3),padding=(2,1))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

也可以看看步幅，不用自己手动实现，直接调用。。

In [7]:
conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

按照之前我在博客里的理解，就是8+2=10,10+（2-3）=9 9/2下取整，就等于4

最后可以看一个稍微复杂点的例子

In [8]:
conv2d=nn.Conv2d(1,1,kernel_size=(3,5),padding=(0,1),stride=(3,4))
comp_conv2d(conv2d,X).shape

torch.Size([2, 2])

如果是这种完全不对称的情况，行和列就要分开计算了

+ 对于行，8行，行上填充为0，行的kernel为3，行的步幅为3，步幅=kernel，
    + 直接8+2*0/3下取整，等于2
+ 对于列，同理，只是计算变成了，(8+2\*1+(4-5))/4=2  (步幅小于卷积核，提前预留二者的差，保证最后一个卷积核可以计算)

> + 一般我们会使用对称的padding和stride以及kernel，因为通常输入的图片是方块。比如28\*28。256\*256。一般数据集都是这么提供的。
> + 当然，如果是自己采集的另算，如果采集的长宽不一样，那么就可以去控制卷积核，步幅和填充有一定的长宽比的形状